# Step 3: Define metrics to use
In general, we want to evaluate 2 things: whether the context retrieved is good, and whether the overall answer is good.

We'll be tracking the following metrics, and for some we'll use the `ragas` library to evaluate. 

## Assessing the context

We need to remember that our RAG system will retrieve multiple contexts. We use multiple metrics here to measure this retrieved context. This can be thought of as an evaluation of the Retrieval part of the system.

- `context_correctness`: Whether the `ground_truth_context` is included in the list of retrieved contexts.
- `ground_truth_context_rank`: The position or rank of the ground_truth_context in the retrieved context. 
- `context_rougel_score`: ROUGE-L score between the ground_truth_context and the top retrieved context.
- `context_precision` (with `ragas`): How relevant the retrieved contexts are to the question (assessed with an LLM).

## Assessing the answer

In contrast to the above, here we evaluate the generated answers themselves. We use two metrics here:

- `faithfulness` (with `ragas`): does the answer use information from the context? (assessed with an LLM)
- `answer_correctness` (with `ragas`): a combination of the following
    - how relevant is the answer to the question? based on cosine similarity of embeddings
    - whether the answer matches with the ground truth, assessed with LLM


In [1]:
import evaluate


def context_correctness(ground_truth_context, contexts):
    """whether the `ground_truth_context` is included in the list of retrieved contexts"""
    return ground_truth_context in contexts


def ground_truth_context_rank(ground_truth_context, contexts):
    """rank of the ground_truth_context in the retrieved contexts, -1 if not found"""
    if ground_truth_context is not None:
        try:
            return contexts.index(ground_truth_context)
        except:
            return -1
    else:
        return -1


def context_rougel_score(ground_truth_context, contexts):
    """ROUGE-L score between the ground_truth_context and the top retrieved context"""
    rouge = evaluate.load('rouge')
    return rouge.compute(predictions=[contexts[0]], references=[ground_truth_context])["rougeL"]


/Users/akashsaravanan/Library/Caches/pypoetry/virtualenvs/genai-bootcamp-4wh1UwyX-py3.12/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


For `ragas`, we can simply call them together as follows:

In [3]:
from datasets import Dataset
from ragas import evaluate as ragas_evaluate
from ragas.metrics import context_precision, faithfulness, answer_correctness
from ragas.llms import LlamaIndexLLMWrapper
from llama_index.llms.openai import OpenAI

def evaluate_w_ragas(item, metrics=[context_precision, faithfulness, answer_correctness]):
    evaluator_llm = LlamaIndexLLMWrapper(OpenAI(model="gpt-4o", temperature=0.0, max_tokens=2048))

    # Format the example into datasets, which ragas takes as inputs
    row_dataset = Dataset.from_pandas(item.to_frame().T)

    # Ragas by default takes in a batch of items and aggregates metrics together
    # So in this example, we need to pass one by one to get individual results.
    # However, it does run faster when you pass all metrics at once.
    ragas_eval_results = ragas_evaluate(row_dataset, metrics=metrics, llm=evaluator_llm, embeddings=None)
    return ragas_eval_results